In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForCausalLM
import re

# Load depression detection model (keep this part unchanged)
model_path = "./depression_bert_model"
depression_tokenizer = BertTokenizer.from_pretrained(model_path)
depression_model = BertForSequenceClassification.from_pretrained(model_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
depression_model.to(device)



# Initialize the model and tokenizer
llm_model, llm_tokenizer = load_offline_llm()

def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.lower().strip()

def predict_depression(text):
    cleaned = clean_text(text)
    inputs = depression_tokenizer(cleaned, return_tensors='pt', truncation=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = depression_model(**inputs)
    
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return probs[0][1].item()

def generate_response(prompt, history):
    system_message = """You are a mental health assistant. Respond with:
    - Empathetic validation
    - Follow-up questions when needed
    - Practical coping suggestions
    - Crisis resources when risk is detected"""
    
    # Format full history using Mistral's template
    messages = [
        {"role": "system", "content": system_message},
        *history,
        {"role": "user", "content": prompt}
    ]
    
    formatted_prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    inputs = llm_tokenizer(formatted_prompt, return_tensors="pt").to(device)
    
    # Generate response
    outputs = llm_model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        do_sample=True
    )
    
    response = llm_tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:], 
        skip_special_tokens=True
    )
    
    return response.strip()

def chat():
    history = []
    print("\n🌱 Mental Health Support Chatbot - Type 'quit' to exit")
    
    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() in ['quit', 'exit']:
            break
            
        # 1. Analyze with depression model
        depression_prob = predict_depression(user_input)
        history.append({"role": "user", "content": user_input})
        
        # 2. Generate context-aware response
        if depression_prob > 0.65:  # High risk
            prompt = (
                f"User shows depression signs (probability: {depression_prob:.2f}). "
                "Provide supportive response with crisis resources."
            )
        elif depression_prob > 0.4:  # Moderate risk
            prompt = (
                f"User may be struggling (probability: {depression_prob:.2f}). "
                "Ask a gentle follow-up question and suggest one coping strategy."
            )
        else:  # Low risk
            prompt = "Acknowledge the message and offer general support."
        
        # Generate and display response
        response = generate_response(prompt, history)
        print(f"\n🌼 Bot: {response}")
        history.append({"role": "assistant", "content": response})
        
        # 3. Store conversation history (optional: save to file)
        # Keep only last 6 messages for context
        history = history[-6:]

def chat():
    history = []
    print("""\n🌱 Mental Health Support Chatbot
    I'm here to listen without judgment. You can:
    - Share as much or little as you like
    - Take breaks anytime with 'quit'
    """)
    
    try:
        while True:
            user_input = input("\nYou: ").strip()
            if user_input.lower() in ['quit', 'exit']:
                print("\nTake care of yourself. You can return anytime.")
                break
                
            # Store the user message in history
            history.append({"role": "user", "content": user_input})
            
            # Analyze input
            prob = predict_depression(user_input)
            
            if prob > 0.4:
                # Generate question 
                question_prompt = f"Based on this statement: '{user_input}', generate a gentle follow-up question"
                question = generate_response(question_prompt, history)
                print(f"\n🌼 Bot: {question}")
                
                # Store the bot's question in history
                history.append({"role": "assistant", "content": question})
                
                # Get response
                answer = input("\nYou: ").strip()
                if answer.lower() in ['quit', 'exit']:
                    print("\nThank you for sharing. Be kind to yourself today.")
                    break
                
                # Store the user's follow-up in history
                history.append({"role": "user", "content": answer})
                
                # Generate advice
                advice_prompt = f"User shared: '{user_input}' then '{answer}'. Provide 2-3 supportive suggestions."
                advice = generate_response(advice_prompt, history)
                print(f"\n🌱 Bot: {advice}")
                
                # Store the bot's advice in history
                history.append({"role": "assistant", "content": advice})
                
                if prob > 0.7:
                    print("\nIf you need immediate support:")
                    print("- National Suicide Prevention Lifeline: 1-800-273-TALK")
                    print("- Crisis Text Line: Text HOME to 741741")
                
            else:
                # Generate acknowledgment
                response = generate_response(f"Acknowledge this feeling: {user_input}", history)
                print(f"\n🌸 Bot: {response}")
                
                # Store the bot's response in history
                history.append({"role": "assistant", "content": response})

            # Continue prompt
            cont = input("\nPress Enter to continue or type 'quit': ").strip()
            if cont.lower() in ['quit', 'exit']:
                print("\nRemember: Progress isn't linear. Be proud you reached out!")
                break

    except KeyboardInterrupt:
        print("\n\nIt's okay to pause. Come back when you're ready.")

if __name__ == "__main__":
    chat()


In [ ]:
import bitsandbytes
print(bitsandbytes.__version__)  # Should output 0.41.1 or higher
